In [1]:
import pandas as pd
import numpy as np

In [2]:
# prompt: connect Google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
Q_df = pd.read_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/test.csv")

In [5]:
A_gemini_df = pd.read_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/processed_financial_analysis_flash2-5_01.csv")

In [6]:
A_gemini_df.domain.unique()

array(['out of domain', 'ethical finance', 'economics', 'stock movement',
       'marketing', 'finance management', 'accounting',
       'financial reporting', 'fixed income'], dtype=object)

In [7]:
A_gemini_df.answer.unique()

array(['D', 'B', 'A', 'Rise', 'C', 'Fall'], dtype=object)

In [8]:
Q_df['query'].iloc[1]

'Answer the question with the appropriate options A, B, C and D. Please respond with the exact answer A, B, C or D only. Do not be verbose or provide extra information. \nQuestion: Who of these is the entrepreneur?\nAnswer Choices: A: Barack Obama, B: James Dyson, C: Damien Hirst, D: Mo Farah \nAnswer:'

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [10]:
# prepare the model input
prompt = "Hello, how are you?"
messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)

model_inputs = tokenizer(text, return_tensors="pt").to(model.device)

outputs = model.generate(
    **model_inputs,
    max_new_tokens=256,
)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
assistant_output = response.split("</think>")[-1].strip()
print(assistant_output)

Hello! I'm just a chatbot, so I don't have feelings, but I'm here and ready to help! How are you? 😊


In [11]:
idx = 0
query = Q_df['query'].iloc[idx]

system_message = """
You are a financial expert. Your job is to answer the questions of given contents strictly, and answer only choices given (choice's description excluded)

Your answer must be structured as:
Answer: <A/B/C/D/E/Fall/Rise>
"""

In [12]:
conversation = [
    {"role": "system", "content": system_message},
    {"role": "user",   "content": query},
]

print(tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True, enable_thinking=True))

<|im_start|>system

You are a financial expert. Your job is to answer the questions of given contents strictly, and answer only choices given (choice's description excluded)

Your answer must be structured as:
Answer: <A/B/C/D/E/Fall/Rise>
<|im_end|>
<|im_start|>user
ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม

คำถาม: ______ สถานที่ทำงานเกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน ตัวอย่างเช่น 'job enrichment' ที่พนักงานได้รับขอบเขตที่ใหญ่ขึ้นในการตัดสินใจว่าจะจัดระเบียบงานของตนอย่างไร หรือ 'job enlargement' ที่พนักงานได้รับมอบหมายงานที่หลากหลายมากขึ้น

ตัวเลือกคำตอบ: A: Re-invigorating, B: Re-flourishing, C: Revitalizing, D: Rehumanizing

คำตอบ:<|im_end|>
<|im_start|>assistant



In [13]:
messages = [
    {"role": "system", "content": system_message},
    {"role": "user",   "content": query},
  ]

full_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
  )

inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=256*8, do_sample=True, temperature=0.01, top_p=0.9, top_k=1)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Only show the assistant's reply (strip the system/user)
assistant_output = response.split("</think>")[-1].strip()
print(assistant_output)

Answer: D


In [14]:
assistant_output.split("Answer: ")[-1].split("\n")[0]

'D'

In [15]:
A_gemini_df.answer.iloc[idx]

'D'

In [16]:
len(Q_df)

499

In [17]:
tokens = tokenizer.encode(query, add_special_tokens=False)
print(f"🔢 Number of tokens: {len(tokens)}")

🔢 Number of tokens: 225


### create reusable function

In [79]:
def extract_answer_from_model(system_message, prompt, tokenizer, model, AnsKey="Answer: "):
    # Construct chat format
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]

    # Convert to full prompt string
    full_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    # Tokenize and run generation
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256*8,
        do_sample=True,
        temperature=0.01,
        top_p=0.9,
        top_k=1
    )

    # Decode output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract assistant reply (after </think>, if present)
    assistant_output = response.split("</think>")[-1].strip()

    if AnsKey == "RETURN: ":
        return assistant_output.split(AnsKey, 1)[-1]

    try:
        answer = assistant_output.split(AnsKey, 1)[-1].split("\n")[0].strip()
    except Exception:
        answer = "?"

    return answer

## try prompt

In [184]:
idx = 3
query = Q_df['query'].iloc[idx]
print(query)

ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้องแม่นยำคือ A, B, C หรือ D เท่านั้น อย่าอธิบายเยิ่นเย้อหรือให้ข้อมูลเพิ่มเติม
คำถาม: หาก GDP ที่แท้จริงปัจจุบันคือ 5000 ดอลลาร์ และ GDP ที่แท้จริงของการจ้างงานเต็มอัตราอยู่ที่ 4000 ดอลลาร์ นโยบายชุดใดต่อไปนี้ที่มีแนวโน้มมากที่สุดที่จะนำพาเศรษฐกิจมาถึงจุดนี้
ตัวเลือกคำตอบ: A: การลดภาษีและอัตราคิดลดที่ต่ำลง, B: การเพิ่มการใช้จ่ายของรัฐบาลและการเพิ่มภาษี, C: การลดภาษีและการขายพันธบัตรในการดำเนินงานในตลาดเปิด, D: การเพิ่มการใช้จ่ายของรัฐบาลและการเพิ่มอัตราคิดลด
คำตอบ:


In [114]:
system_message_clean = """
You are a helpful assistant. Your task is to remove all task descriptions or instructions from an input text file. This includes any lines that:
- Tell the user what to do (e.g., "ตอบคำถาม", "เลือกคำตอบ", "คำชี้แจง", etc.)
- Explain how to answer or what format to use
- Provide general instructions unrelated to the question content

Keep only:
- The core question text
- The answer choices (if any), labeled A, B, C, D, etc.

Do not add anything to the output.
RETURN: <only the cleaned question and choices.>
"""
system_message_clean2Eng = """
You are a helpful assistant. Your task is to process an input text that contains one or more multiple-choice questions. For each question:

1. Remove all task descriptions or instructions. This includes any lines that:
   - Instruct the user on what to do (e.g., "ตอบคำถาม", "เลือกคำตอบ", "คำชี้แจง", etc.)
   - Explain how to answer or what format to use
   - Provide general guidelines unrelated to the actual question

2. Keep only:
   - The core question content
   - The answer choices (if any), labeled A, B, C, D, etc.

3. Then, translate the cleaned question and answer choices into clear, natural English.

Do not add any commentary, formatting instructions, or extra text.

RETURN: <only the cleaned and translated question and choices in English.>
"""

clean_query = extract_answer_from_model(system_message_clean2Eng, query, tokenizer, model, AnsKey="RETURN: ")
clean_query

'What policy is most likely to lead the economy to the current state if the actual GDP is $5,000 and the full employment GDP is $4,000?  \nA: Tax cuts and lower discount rates  \nB: Increased government spending and higher taxes  \nC: Tax cuts and selling bonds in the open market  \nD: Increased government spending and higher discount rates'

In [115]:
system_message_9D = """
You are an expert financial AI assistant. Your task is to analyze a given text and classify it into exactly one of the following financial domains:

9 categories of Domain list:
- out of domain
- ethical finance
- economics
- stock movement
- marketing
- finance management
- accounting
- financial reporting
- fixed income

Please analyze the meaning, terminology, and financial context of the input and select the most appropriate domain **based only on the content**.

Your response must be in the following format:
Domain: <one of the 9 categories above>

Do not provide any explanation or reasoning. Output only the domain label.
"""
extract_answer_from_model(system_message_9D, clean_query, tokenizer, model, AnsKey="Domain: ")

'Economics'

In [116]:
system_message_out_of_domain = """
You are a general-purpose AI assistant. Your task is to answer multiple-choice questions on any topic, including general knowledge, science, history, language, politics, or other subjects beyond finance.

You must answer strictly based on the content of the question. Choose only one of the provided options and ignore the explanation or description of each choice.

Your answer must follow this exact format:
Answer: <A/B/C/D/E/Fall/Rise>

Do not include any reasoning, commentary, or additional text.
"""

system_message_ethical_finance = """
You are a financial expert specializing in ethical finance. Your task is to analyze and answer multiple-choice questions related to ethical principles in financial decision-making, corporate responsibility, sustainable investing, stakeholder impact, and related topics.

You must answer strictly based on the content of the question. Choose only one of the provided choices and ignore the explanation or description of each option.

Your answer must follow this exact format:
Answer: <A/B/C/D/E/Fall/Rise>

Do not include any reasoning, justification, or additional text.
"""

system_message_economics = """
You are an expert in economics. Your task is to analyze and answer multiple-choice questions related to microeconomics, macroeconomics, economic theory, market structures, economic indicators, supply and demand, inflation, fiscal and monetary policy, and other related topics.

You must answer strictly based on the content of the question. Choose only one of the provided choices and ignore the explanation or description of each option.

Your answer must follow this exact format:
Answer: <A/B/C/D/E/Fall/Rise>

Do not include any reasoning, explanation, or additional text.
"""

system_message_stock_movement = """
You are an expert in stock market analysis. Your task is to answer multiple-choice questions related to stock price movement, technical indicators, investor sentiment, market reactions, news impact, and related concepts.

You must answer strictly based on the content of the question. Choose only one of the provided choices and ignore the explanation or description of each option.

Your answer must follow this exact format:
Answer: <Fall/Rise>

Use “Fall” if the correct answer implies a downward movement in stock price, or “Rise” if it implies an upward movement. Otherwise, use A, B, C, D, or E based on the choices.

Do not include any reasoning, explanation, or additional text.
"""

system_message_marketing = """
You are an expert in marketing. Your task is to answer multiple-choice questions related to marketing principles, consumer behavior, branding, market segmentation, advertising, pricing strategies, promotion, digital marketing, and other related topics.

You must answer strictly based on the content of the question. Choose only one of the provided choices and ignore the explanation or description of each option.

Your answer must follow this exact format:
Answer: <A/B/C/D/E/Fall/Rise>

Do not include any reasoning, explanation, or additional text.
"""

system_message_finance_management = """
You are an expert in finance management. Your task is to answer multiple-choice questions related to financial planning, budgeting, capital structure, investment decisions, risk management, time value of money, cost of capital, and related financial management topics.

You must answer strictly based on the content of the question. Choose only one of the provided choices and ignore the explanation or description of each option.

Your answer must follow this exact format:
Answer: <A/B/C/D/E/Fall/Rise>

Do not include any reasoning, explanation, or additional text.
"""

system_message_accounting = """
You are an expert in accounting. Your task is to answer multiple-choice questions related to financial accounting, managerial accounting, bookkeeping, journal entries, ledgers, accruals, depreciation, cost accounting, and other accounting-related topics.

You must answer strictly based on the content of the question. Choose only one of the provided choices and ignore the explanation or description of each option.

Your answer must follow this exact format:
Answer: <A/B/C/D/E/Fall/Rise>

Do not include any reasoning, explanation, or additional text.
"""

system_message_financial_reporting = """
You are an expert in financial reporting. Your task is to answer multiple-choice questions related to financial statements, IFRS, GAAP, income statements, balance sheets, cash flow statements, disclosure requirements, consolidation, and other financial reporting topics.

You must answer strictly based on the content of the question. Choose only one of the provided choices and ignore the explanation or description of each option.

Your answer must follow this exact format:
Answer: <A/B/C/D/E/Fall/Rise>

Do not include any reasoning, explanation, or additional text.
"""

system_message_fixed_income = """
You are an expert in fixed income. Your task is to answer multiple-choice questions related to bonds, interest rates, yield curves, duration, convexity, credit risk, coupon structures, inflation-linked securities, and other fixed income instruments and concepts.

You must answer strictly based on the content of the question. Choose only one of the provided choices and ignore the explanation or description of each option.

Your answer must follow this exact format:
Answer: <A/B/C/D/E/Fall/Rise>

Use “Fall” or “Rise” only if the question clearly implies a directional movement in interest rates, bond prices, or yields. Otherwise, choose one of A, B, C, D, or E.

Do not include any reasoning, explanation, or additional text.
"""

print(extract_answer_from_model(system_message_out_of_domain, clean_query, tokenizer, model))
print(extract_answer_from_model(system_message_ethical_finance, clean_query, tokenizer, model))
print(extract_answer_from_model(system_message_economics, clean_query, tokenizer, model))
print(extract_answer_from_model(system_message_stock_movement, clean_query, tokenizer, model))
print(extract_answer_from_model(system_message_marketing, clean_query, tokenizer, model))
print(extract_answer_from_model(system_message_finance_management, clean_query, tokenizer, model))
print(extract_answer_from_model(system_message_accounting, clean_query, tokenizer, model))
print(extract_answer_from_model(system_message_financial_reporting, clean_query, tokenizer, model))
print(extract_answer_from_model(system_message_fixed_income, clean_query, tokenizer, model))

B
A
B
A
A
A
A
A
A


In [185]:
A_gemini_df['answer'][idx]

'A'

In [190]:
from collections import Counter

def process_question_with_domain_routing(query):
    def extract(system_message, prompt, AnsKey):
        return extract_answer_from_model(system_message, prompt, tokenizer, model, AnsKey=AnsKey)

    def majority_vote(candidates):
        counts = Counter(candidates)
        return counts.most_common(1)[0][0]

    # Step 1: Clean and translate
    clean_query = extract(system_message_clean2Eng, query, AnsKey="RETURN: ")
    print("    Raw Query    :", query[:400])
    print("Cleaned Query    :", clean_query[:400])

    # Step 2: Classify domain (for reference or logging)
    domain = extract(system_message_9D, clean_query, AnsKey="Domain: ")
    print("Classified Domain:", domain)

    # Step 3: Route to all domain experts and collect their votes
    domain_map = {
        "out of domain": system_message_out_of_domain,
        "ethical finance": system_message_ethical_finance,
        "economics": system_message_economics,
        "stock movement": system_message_stock_movement,
        "marketing": system_message_marketing,
        "finance management": system_message_finance_management,
        "accounting": system_message_accounting,
        "financial reporting": system_message_financial_reporting,
        "fixed income": system_message_fixed_income,
    }

    if domain.lower() in "stock movement" or "stock" in domain.lower():
        final_out = extract(domain_map["stock movement"], query, AnsKey="Answer: ")
        member_out = [final_out]
    else:
        member_out = []
        for key, system_msg in domain_map.items():
            answer = extract(system_msg, clean_query, AnsKey="Answer: ")
            member_out.append(answer)
        final_out = majority_vote(member_out)
    print("Final Answer     :", final_out)
    print("Member Votes     :", member_out)

    return {
        "cleaned_query": clean_query,
        "domain": domain,
        "answer": final_out
    }

In [186]:
answer = process_question_with_domain_routing(query)

    Raw Query    : ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้องแม่นยำคือ A, B, C หรือ D เท่านั้น อย่าอธิบายเยิ่นเย้อหรือให้ข้อมูลเพิ่มเติม
คำถาม: หาก GDP ที่แท้จริงปัจจุบันคือ 5000 ดอลลาร์ และ GDP ที่แท้จริงของการจ้างงานเต็มอัตราอยู่ที่ 4000 ดอลลาร์ นโยบายชุดใดต่อไปนี้ที่มีแนวโน้มมากที่สุดที่จะนำพาเศรษฐกิจมาถึงจุดนี้
ตัวเลือกคำตอบ: A: การลดภาษีและอัตราคิดลดที่ต่ำลง, B: การเพิ่มการใช้จ่ายขอ
Cleaned Query    : What policy is most likely to lead the economy to the current state if the actual GDP is $5,000 and the full employment GDP is $4,000?  
A: Tax cuts and lower discount rates  
B: Increased government spending and higher taxes  
C: Tax cuts and selling bonds in the open market  
D: Increased government spending and higher discount rates
Classified Domain: Economics
Final Answer     : A
Member Votes     : ['B', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'A']


### Loop Agents

In [191]:
from tqdm import tqdm

ans_train = []
gt_train = []
out_train = []

for idx in range(len(Q_df)):
    prompt = Q_df['query'][idx]

    print(f"sample : {idx+1}/{len(Q_df)}")
    print("ground_truth :", A_gemini_df['answer'][idx])

    answer = process_question_with_domain_routing(prompt)
    print("----------------------------------------------------")

    ans_train.append(answer["answer"])
    gt_train.append(A_gemini_df['answer'][idx])
    out_train.append(answer)

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
Cleaned Query    : What is a paradigm?  
A: A theoretical assumption  
B: An illustration  
C: A theoretical model  
D: An example serving as a model
Classified Domain: out of domain
Final Answer     : C
Member Votes     : ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
----------------------------------------------------
sample : 167/499
ground_truth : Fall
    Raw Query    : By reviewing the data and tweets, can we predict if the closing price of $mmm will go upwards or downwards at 2018-01-19? Please indicate either Rise or Fall.
Context: date,open,high,low,close,adj-close,inc-5,inc-10,inc-15,inc-20,inc-25,inc-30
2018-01-04,-0.7,0.3,-0.9,1.3,1.3,-1.0,-1.2,-0.8,-0.6,-0.4,-0.7
2018-01-05,-0.8,0.1,-1.2,0.8,0.8,-1.4,-1.8,-1.6,-1.4,-1.1,-1.4
2018-01-08,-0.2,0.5,-0.3,-0.3,
Cleaned Query    : What is the predicted direction of the closing price of $mmm on 2018-01-19?  
A) Rise  
B) Fall
Classified Domain: Stock movement
Final Answer   

In [ ]:
#ans_train = [x.split(".")[0] for x in ans_train]

In [192]:
import numpy as np
np.unique(ans_train)

array(['A', 'B', 'C', 'D', 'Fall', 'Rise', 'ขึ้น'], dtype='<U4')

In [193]:
'''

import numpy as np

# Normalization function
def normalize_answer(ans):
    ans_ = ans.strip().upper()

    if 'FALL' in ans_ or 'DOWN' in ans_ or 'ลด' in ans_ or 'DECREASE' in ans:
        return 'Fall'
    elif 'INCREASE' in ans_ or 'UP' in ans_:
        return 'Rise'

    return ans

# Apply normalization
normalized = np.array([normalize_answer(a) for a in ans_train])

# Show unique cleaned answers
cleaned = np.unique(normalized)
print(cleaned)
'''

"\n\nimport numpy as np\n\n# Normalization function\ndef normalize_answer(ans):\n    ans_ = ans.strip().upper()\n\n    if 'FALL' in ans_ or 'DOWN' in ans_ or 'ลด' in ans_ or 'DECREASE' in ans:\n        return 'Fall'\n    elif 'INCREASE' in ans_ or 'UP' in ans_:\n        return 'Rise'\n    \n    return ans\n\n# Apply normalization\nnormalized = np.array([normalize_answer(a) for a in ans_train])\n\n# Show unique cleaned answers\ncleaned = np.unique(normalized)\nprint(cleaned)\n"

In [194]:
ans_train_ = ans_train

In [195]:
# Calculate accuracy
correct = sum(p == g for p, g in zip(ans_train_, ans_train))
accuracy = correct / len(gt_train)

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 100.00%


In [196]:
# Calculate accuracy
correct = sum(p == g for p, g in zip(ans_train_, gt_train))
accuracy = correct / len(gt_train)

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 76.15%


In [197]:
submission_df = pd.read_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/submission.csv")
submission_df

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,NaN
4,bcca13bc-2675-4645-82cc-7e4c412ed294,NaN
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,NaN
495,73c720b5-1101-4790-af52-3366823e1d32,NaN
496,357db18f-d872-416e-a07f-753099853d9c,NaN
497,2d8b1419-1c46-4e83-892a-081fb417de38,NaN


In [198]:
submission_df['answer'] = ans_train_
submission_df

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,A
4,bcca13bc-2675-4645-82cc-7e4c412ed294,Rise
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,D
495,73c720b5-1101-4790-af52-3366823e1d32,A
496,357db18f-d872-416e-a07f-753099853d9c,D
497,2d8b1419-1c46-4e83-892a-081fb417de38,Rise


In [199]:
submission_df.to_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/submission/submission_Qwen_Qwen3_8B_unthink_multiagent2.csv", index=False)

In [200]:
think_df = pd.read_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/submission/ubmission_Qwen_Qwen3_8B_think_raw.csv")

In [201]:
think_df

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,<A/B/C/D/E/Fall/Rise>
4,bcca13bc-2675-4645-82cc-7e4c412ed294,Rise
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,B
495,73c720b5-1101-4790-af52-3366823e1d32,A
496,357db18f-d872-416e-a07f-753099853d9c,D
497,2d8b1419-1c46-4e83-892a-081fb417de38,<A/B/C/D/E/Fall/Rise>


In [202]:
submission_mix2_df = think_df.copy()
mask = submission_mix2_df.answer == "<A/B/C/D/E/Fall/Rise>"
submission_mix2_df.loc[mask] = submission_df.loc[mask]

In [203]:
submission_mix2_df

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,A
4,bcca13bc-2675-4645-82cc-7e4c412ed294,Rise
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,B
495,73c720b5-1101-4790-af52-3366823e1d32,A
496,357db18f-d872-416e-a07f-753099853d9c,D
497,2d8b1419-1c46-4e83-892a-081fb417de38,Rise


In [205]:
# Calculate accuracy
correct = sum(p == g for p, g in zip(submission_mix2_df.answer, gt_train))
accuracy = correct / len(gt_train)

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 82.77%


In [206]:
submission_mix2_df.to_csv("/content/drive/MyDrive/SuperAIRun/Financial Analysis Agent Hackathon/financial-analysis-agent-data/submission/ubmission_Qwen_Qwen3_8B_think_mix2.csv", index=False)